<a href="https://colab.research.google.com/github/Fermasango28/Fermasango28/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.metrics import confusion_matrix


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/ML/SEMANA2/train.csv")
df.set_index('PassengerId',inplace = True)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [4]:
df = df.drop_duplicates()
df.isna().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [5]:
df = df.drop(columns=['Cabin','Name','Ticket'])

In [6]:
df = df.dropna()

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Survived,712.0,0.404494,0.491139,0.00,0.00,0.00000,1.0,1.0000
Pclass,712.0,2.240169,0.836854,1.00,1.00,2.00000,3.0,3.0000
Age,712.0,29.642093,14.492933,0.42,20.00,28.00000,38.0,80.0000
SibSp,712.0,0.514045,0.930692,0.00,0.00,0.00000,1.0,5.0000
Parch,712.0,0.432584,0.854181,0.00,0.00,0.00000,1.0,6.0000
Fare,712.0,34.567251,52.938648,0.00,8.05,15.64585,33.0,512.3292


In [8]:
df[df['Age']<1]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
79,1,2,male,0.83,0,2,29.0000,S
306,1,1,male,0.92,1,2,151.5500,S
470,1,3,female,0.75,2,1,19.2583,C
645,1,3,female,0.75,2,1,19.2583,C
756,1,2,male,0.67,1,1,14.5000,S
804,1,3,male,0.42,0,1,8.5167,C
832,1,2,male,0.83,1,1,18.7500,S


In [9]:
y = df['Survived']
x = df.drop(columns=['Survived'])

In [10]:
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

one_hot_encoder = OneHotEncoder()
standard_scaler = StandardScaler()

imputer_num = SimpleImputer(strategy="mean")
imputer_cat = SimpleImputer(strategy="most_frequent")

col_transformer = make_column_transformer((make_pipeline(imputer_num,standard_scaler), num_selector),(make_pipeline(imputer_cat,one_hot_encoder), cat_selector),remainder = 'passthrough')

col_transformer.fit(x)
aux=col_transformer.transform(x)
X_procesado=pd.DataFrame(aux,columns=col_transformer.get_feature_names_out())
X_procesado.info()
X_procesado

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   pipeline-1__Pclass      712 non-null    float64
 1   pipeline-1__Age         712 non-null    float64
 2   pipeline-1__SibSp       712 non-null    float64
 3   pipeline-1__Parch       712 non-null    float64
 4   pipeline-1__Fare        712 non-null    float64
 5   pipeline-2__Sex_female  712 non-null    float64
 6   pipeline-2__Sex_male    712 non-null    float64
 7   pipeline-2__Embarked_C  712 non-null    float64
 8   pipeline-2__Embarked_Q  712 non-null    float64
 9   pipeline-2__Embarked_S  712 non-null    float64
dtypes: float64(10)
memory usage: 55.8 KB


,pipeline-1__Pclass,pipeline-1__Age,pipeline-1__SibSp,pipeline-1__Parch,pipeline-1__Fare,pipeline-2__Sex_female,pipeline-2__Sex_male,pipeline-2__Embarked_C,pipeline-2__Embarked_Q,pipeline-2__Embarked_S
0,0.908600,-0.527669,0.522511,-0.506787,-0.516380,0.0,1.0,0.0,0.0,1.0
1,-1.482983,0.577094,0.522511,-0.506787,0.694046,1.0,0.0,1.0,0.0,0.0
2,0.908600,-0.251478,-0.552714,-0.506787,-0.503620,1.0,0.0,0.0,0.0,1.0
3,-1.482983,0.369951,0.522511,-0.506787,0.350326,1.0,0.0,0.0,0.0,1.0
4,0.908600,0.369951,-0.552714,-0.506787,-0.501257,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
707,0.908600,0.646142,-0.552714,5.350885,-0.102875,1.0,0.0,0.0,1.0,0.0
708,-0.287191,-0.182430,-0.552714,-0.506787,-0.407687,0.0,1.0,0.0,0.0,1.0
709,-1.482983,-0.734812,-0.552714,-0.506787,-0.086335,1.0,0.0,0.0,0.0,1.0
710,-1.482983,-0.251478,-0.552714,-0.506787,-0.086335,0.0,1.0,1.0,0.0,0.0


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_procesado, y, test_size=0.3,random_state=6)

In [12]:
model = Sequential()
model.add(Dense(200,input_dim = X_train.shape[1], activation = 'relu'))
model.add(Dense(100, activation = 'relu',kernel_regularizer=l2(0.05)))
model.add(Dense(50, activation = 'relu',kernel_regularizer=l2(0.05)))
model.add(Dense(25, activation = 'relu',kernel_regularizer=l2(0.05)))
model.add(Dense(1,activation = "sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               2200      
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dense_3 (Dense)             (None, 25)                1275      
                                                                 
 dense_4 (Dense)             (None, 1)                 26        
                                                                 
Total params: 28651 (111.92 KB)
Trainable params: 28651 (111.92 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
model.compile(loss = 'bce', optimizer = 'adam', metrics=['accuracy'])
history = model.fit(X_train, y_train,validation_data = (X_test, y_test), epochs=100)

Epoch 1/100
16/16 [==============================] - 2s 22ms/step - loss: 11.0963 - accuracy: 0.7590 - val_loss: 9.5133 - val_accuracy: 0.7477
Epoch 2/100
16/16 [==============================] - 0s 5ms/step - loss: 8.3788 - accuracy: 0.7731 - val_loss: 7.1718 - val_accuracy: 0.7477
Epoch 3/100
16/16 [==============================] - 0s 6ms/step - loss: 6.3084 - accuracy: 0.8032 - val_loss: 5.4168 - val_accuracy: 0.7430
Epoch 4/100
16/16 [==============================] - 0s 9ms/step - loss: 4.7712 - accuracy: 0.8112 - val_loss: 4.1266 - val_accuracy: 0.7664
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 3.6397 - accuracy: 0.8233 - val_loss: 3.1869 - val_accuracy: 0.7477
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 2.8173 - accuracy: 0.8213 - val_loss: 2.4960 - val_accuracy: 0.7757
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 2.2083 - accuracy: 0.8353 - val_loss: 1.9921 - val_accuracy: 0.7664
Epoch 8/100

In [14]:
df1 = pd.read_csv("/content/drive/MyDrive/ML/SEMANA2/test.csv")
df1.set_index('PassengerId',inplace = True)
df1.info()
df1

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    object 
 7   Fare      417 non-null    float64
 8   Cabin     91 non-null     object 
 9   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [15]:
df2= pd.read_csv("/content/drive/MyDrive/ML/SEMANA2/gender_submission.csv")
df2.set_index('PassengerId',inplace = True)

In [16]:
df_test=pd.concat([df1,df2],axis=1)

In [17]:
df_test

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
PassengerId,,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0


In [18]:
df_test = df_test.drop_duplicates()
df_test.isna().sum()
df_test = df_test.drop(columns=['Cabin','Name','Ticket'])

In [19]:
df_test = df_test.dropna()
df_test.describe()

,Pclass,Age,SibSp,Parch,Fare,Survived
count,331.000000,331.000000,331.000000,331.000000,331.000000,331.000000
mean,2.141994,30.181269,0.483384,0.398792,40.982087,0.383686
std,0.846251,14.104573,0.875004,0.811582,61.228558,0.487019
min,1.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,21.000000,0.000000,0.000000,8.050000,0.000000
50%,2.000000,27.000000,0.000000,0.000000,16.000000,0.000000
75%,3.000000,39.000000,1.000000,1.000000,40.633350,1.000000
max,3.000000,76.000000,8.000000,6.000000,512.329200,1.000000


In [20]:
y = df_test['Survived']
x = df_test.drop(columns=['Survived'])

In [21]:
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

one_hot_encoder = OneHotEncoder()
standard_scaler = StandardScaler()

imputer_num = SimpleImputer(strategy="mean")
imputer_cat = SimpleImputer(strategy="most_frequent")

col_transformer = make_column_transformer((make_pipeline(imputer_num,standard_scaler), num_selector),(make_pipeline(imputer_cat,one_hot_encoder), cat_selector),remainder = 'passthrough')

col_transformer.fit(x)
aux=col_transformer.transform(x)
X_procesado=pd.DataFrame(aux,columns=col_transformer.get_feature_names_out())
X_procesado.info()
X_procesado

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   pipeline-1__Pclass      331 non-null    float64
 1   pipeline-1__Age         331 non-null    float64
 2   pipeline-1__SibSp       331 non-null    float64
 3   pipeline-1__Parch       331 non-null    float64
 4   pipeline-1__Fare        331 non-null    float64
 5   pipeline-2__Sex_female  331 non-null    float64
 6   pipeline-2__Sex_male    331 non-null    float64
 7   pipeline-2__Embarked_C  331 non-null    float64
 8   pipeline-2__Embarked_Q  331 non-null    float64
 9   pipeline-2__Embarked_S  331 non-null    float64
dtypes: float64(10)
memory usage: 26.0 KB


,pipeline-1__Pclass,pipeline-1__Age,pipeline-1__SibSp,pipeline-1__Parch,pipeline-1__Fare,pipeline-2__Sex_female,pipeline-2__Sex_male,pipeline-2__Embarked_C,pipeline-2__Embarked_Q,pipeline-2__Embarked_S
0,1.015426,0.306657,-0.553272,-0.492120,-0.542281,0.0,1.0,0.0,1.0,0.0
1,1.015426,1.194236,0.591310,-0.492120,-0.555844,1.0,0.0,0.0,0.0,1.0
2,-0.168046,2.259331,-0.553272,-0.492120,-0.511885,0.0,1.0,0.0,1.0,0.0
3,1.015426,-0.225890,-0.553272,-0.492120,-0.528651,0.0,1.0,0.0,0.0,1.0
4,1.015426,-0.580922,0.591310,0.741907,-0.469357,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
326,1.015426,-1.930042,0.591310,0.741907,-0.445026,1.0,0.0,0.0,0.0,1.0
327,-1.351518,0.484173,0.591310,-0.492120,0.801785,1.0,0.0,0.0,1.0,0.0
328,1.015426,-0.154884,-0.553272,-0.492120,-0.543168,1.0,0.0,0.0,0.0,1.0
329,-1.351518,0.626186,-0.553272,-0.492120,1.110932,1.0,0.0,1.0,0.0,0.0


In [29]:
y_pred = model.predict(X_procesado).round()
matriz = confusion_matrix(y,y_pred)
TP = matriz[0][0]
TN = matriz[1][1]
FP = matriz[0][1]
FN = matriz[1][0]
print("Accuracy: ",(TP+TN)/(TP+TN+FP+FN))

11/11 [==============================] - 0s 3ms/step
Accuracy:  0.8761329305135952
